In [ ]:
%load_ext autoreload 
%autoreload 2
from hig_data.vg2 import VGGraphPrecomputedDataset

# path,                       # Path to the precomputed h5 file 
#                  vocab_path,                 # Path to the vocab h5 file
#                  split_path,                 # Path to dataset ID splits

vg = VGGraphPrecomputedDataset(path='/home/rfsm2/rds/hpc-work/datasets/vg/all_vg_graph.h5',
                         vocab_path='/home/rfsm2/rds/hpc-work/datasets/vg/all_vg_clip_vocab.h5',
                        split_path='/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/vg_splits.json',)


In [ ]:
import h5py
with h5py.File(vg.path, 'r') as hdf:
        group = hdf[str(vg._data_fnames[1])]
        print(list(group.keys()))
        image = group['image'][:]

In [ ]:
from hig_data.visualisation import plot_array_images
from training.encoders import StabilityVAEEncoder
import numpy as np
import torch
vae = StabilityVAEEncoder(batch_size=8, vae_name='stabilityai/sdxl-vae')
ground_truth = vae.decode(vae.encode_latents(torch.tensor(image[np.newaxis,...]).to('cuda')))
plot_array_images(ground_truth.cpu()) 



In [ ]:
import zipfile

latents = zipfile.ZipFile('/home/rfsm2/rds/hpc-work/datasets/vg/all_vg_512-sd.zip')
files = latents.filelist
with latents.open(files[0], 'r') as f:
    img = np.load(f)


In [ ]:
from hig_data.visualisation import plot_array_images
from training.encoders import StabilityVAEEncoder
import numpy as np
import torch
vae = StabilityVAEEncoder(batch_size=8, vae_name='stabilityai/sdxl-vae')
ground_truth = vae.decode(vae.encode_latents(torch.tensor(img[np.newaxis,...]).to('cuda')))
plot_array_images(ground_truth.cpu()) 



In [ ]:
vg[0].metadata()

In [ ]:
from training.networks_edm2_hignn_control import Precond
import torch
precond = Precond(64, 4, gnn_metadata = vg[0].metadata()).cuda()


In [ ]:
# precond
blank_image = torch.zeros((1, 192, 64, 64)).cuda()
out, new_graph = precond.unet.hignn(blank_image, vg[2].clone().cuda())
out.shape

In [ ]:
from hig_data.visualisation import plot_array_images
plot_array_images(out.detach().cpu()[:,3:6]) 


In [ ]:
from hig_data.utils import DataLoader
dls = DataLoader(vg, batch_size=8, subsample=False)

In [ ]:
graph_batch = next(iter(dls))
graph_batch

In [ ]:
from hig_data.visualisation import plot_array_images
from training.encoders import StabilityVAEEncoder
vae = StabilityVAEEncoder(batch_size=8,)
ground_truth = vae.decode(vae.encode_latents(graph_batch.image.to('cuda')))
plot_array_images(ground_truth.cpu()) 

In [ ]:
%load_ext autoreload 
%autoreload 2
from hig_data.vg import VgSceneGraphDataset

root = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/'
vg = VgSceneGraphDataset(vocab_json=root+'vocab.json',
                         h5_path=root+'val.h5',
                        #  image_dir='/home/rfsm2/rds/hpc-work/datasets/vg/all_vg_512',
                        image_dir=root+'images',
                         image_size=512,)


In [ ]:
import h5py
path = '/home/rfsm2/rds/hpc-work/datasets/vg/all_vg_clip_vocab.h5'
with h5py.File(path, 'w') as hdf:
    vocab = hdf.keys()

In [ ]:
vocab

In [ ]:
index = 650
objs, obj_bbox, triples, img_path, img = vg[index]


In [ ]:

print([vg.vocab['object_idx_to_name'][o] for o in objs])
print(vg.data['attributes_per_object'][index], vg.data['object_attributes'][index])
# print(vg.data['object_attributes'][index])

In [ ]:
img.shape

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
def visualize_bounding_boxes(image: np.ndarray, bboxes: np.ndarray, labels: list, bbox_color='red', text_color='white'):
    """
    Visualizes bounding boxes on a numpy array image.
    
    Arguments:
        image: 2D NumPy array representing the image.
        bboxes: Bounding boxes as an Nx4 numpy array where each row is [xmin, ymin, xmax, ymax].
                Coordinates are expected to be in 0-1 normalized format.
        bbox_color: The color for the bounding boxes (default is red).
    """
    fig, ax = plt.subplots(1)
    ax.imshow(image,)
    
    # Image dimensions
    img_h, img_w, *_ = image.shape
    
    # Iterate over bounding boxes and draw them with labels
    for bbox, label in zip(bboxes, labels):

        # Denormalize bbox coordinates back to pixel values
        # xmin, ymin, xmax, ymax = bbox * np.array([img_w, img_h, img_w, img_h])
        xmin, ymin, xmax, ymax = bbox
        xmin, ymin, xmax, ymax = bbox * np.array([img_w, img_h, img_w, img_h])
        width = xmax - xmin
        height = ymax - ymin

        # xmin, ymin, width, height = bbox
        
        # Create a rectangle patch
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor=bbox_color, facecolor='none')
        ax.add_patch(rect)
        
        # Add label text
        ax.text(xmin, ymin - 5, label, color=text_color, fontsize=8, bbox=dict(facecolor=bbox_color, alpha=0.5, pad=1))

    plt.show()

In [ ]:
from PIL import Image
import numpy as np
# objs, obj_bbox, triples, img_path, img = vg[index]
# img = np.array(Image.open(img_path.replace('png', 'jpg')))
triples
obj_vocab = vg.vocab['object_idx_to_name']
pred_vocb = vg.vocab['pred_idx_to_name']
print(len(triples), len(objs))
for s,p,o in triples:
    print(obj_vocab[objs[s]], pred_vocb[p], obj_vocab[objs[o]])
    
im = np.array(img).transpose(1,2,0)
# print(img.shape)
labels = [vg.vocab['object_idx_to_name'][o] for o in objs]
# print(labels, obj_bbox[-1])
visualize_bounding_boxes(im, obj_bbox, labels)

In [ ]:
# import numpy as np
# index = 1000
# obj_idxs = np.array(range(vg.data['objects_per_image'][index].item()))

# objs = []
# obj_idx_mapping = {}
# for i, obj_idx in enumerate(obj_idxs):
#     objs.append(vg.data['object_names'][index, obj_idx].item())
#     obj_idx_mapping[obj_idx] = i

# print([obj_vocab[i] for i in objs])

# triples = []
# print('relationships_per_image: ', vg.data['relationships_per_image'][index].item())
# for r_idx in range(vg.data['relationships_per_image'][index].item()):
#     s = vg.data['relationship_subjects'][index, r_idx].item()
#     p = vg.data['relationship_predicates'][index, r_idx].item()
#     o = vg.data['relationship_objects'][index, r_idx].item()
#     s = obj_idx_mapping.get(s, None)
#     o = obj_idx_mapping.get(o, None)
#     if s is not None and o is not None:
#         triples.append([s, p, o])

# obj_vocab = vg.vocab['object_idx_to_name']
# pred_vocb = vg.vocab['pred_idx_to_name']
# for s,p,o in triples:
#     print(obj_vocab[objs[s]], pred_vocb[p], obj_vocab[objs[o]])


In [ ]:
from torchvision.transforms import CenterCrop
import torch
import numpy as np
index = 1000
objs, obj_bbox, triples, img_path = vg[index]
HH, WW = np.array(vg.data['image_widths'][index]), np.array(vg.data['image_heights'][index])
print(HH,WW)
out, valid = vg.center_crop_bboxes_with_filter(WW, HH, obj_bbox, 0.01)
out = np.array(out).astype(np.int32)

In [ ]:
import numpy as np
index = 1000
WW, HH = np.array(vg.data['image_widths'][index]), np.array(vg.data['image_heights'][index])
from PIL import Image
print(WW,HH)
# im = Image.open(img_path.replace('png', 'jpg'))
im = Image.open(img_path)
im

In [ ]:
import torchvision
import torch
WW, HH = np.array(vg.data['image_widths'][index]), np.array(vg.data['image_heights'][index])
obj_bbox = torchvision.transforms.CenterCrop(size=(512,512))(torch.tensor(obj_bbox), spatial_size=(WW,HH))
obj_bbox.shape

In [ ]:
import numpy as np
obj_idxs = np.array(range(vg.data['objects_per_image'][index].item()))
print(obj_idxs)

In [ ]:
# import numpy as np
# attributes_per_object = vg.data['attributes_per_object'][index].item()
# object_attributes = vg.data['object_attributes'][index].item()
# print(attributes_per_object, object_attributes)

In [ ]:
from PIL import Image
im = Image.open(img_path)
print([obj_vocab[i] for i in objs])
im

In [ ]:
from PIL import Image
index=1
objs, obj_bbox, triples, img_path = vg[index]
obj_vocab = vg.vocab['object_idx_to_name']
pred_vocb = vg.vocab['pred_idx_to_name']
# for r_idx in range(vg.data['relationships_per_image'][index].item()):
#     s = vg.data['relationship_subjects'][index, r_idx].item()
#     p = vg.data['relationship_predicates'][index, r_idx].item()
#     o = vg.data['relationship_objects'][index, r_idx].item()
for s,p,o in triples:
    print(obj_vocab[objs[s]], pred_vocb[p], obj_vocab[objs[o]])

im = Image.open(img_path)
im

In [ ]:
triples

In [ ]:
vg.vocab.keys()

In [ ]:
 # get all object idxs
import numpy as np
import torch
import os
import PIL
index=300
raw_path = vg.image_paths[index].decode('utf-8')
raw_base_name = os.path.splitext(os.path.basename(raw_path))[0]
img_path = os.path.join(vg.image_dir, os.path.basename(raw_base_name)+'.png')
print(img_path)
WW, HH = np.array(vg.data['image_widths'][index]), np.array(vg.data['image_heights'][index])
obj_idxs = np.array(range(vg.data['objects_per_image'][index].item()))
n_objs = len(obj_idxs) 
print(f'n objs in img {n_objs}')
is_valid_obj = [True for _ in range(n_objs)]
obj_bbox = np.array(vg.data['object_boxes'][index][:n_objs])
obj_bbox, is_valid_obj = vg.filter_invalid_bbox(H=HH, W=WW, bbox=obj_bbox, is_valid_bbox=is_valid_obj)
obj_bbox = obj_bbox[is_valid_obj]
obj_bbox, is_valid_bb = vg.center_crop_bboxes_with_filter(WW, HH, bboxes=obj_bbox) # crop and resize w.r.t to original image size
obj_names =  vg.data['object_names'][index][is_valid_bb]

# create mapping
obj_idxs = obj_idxs[is_valid_bb]
n_objs = len(obj_idxs) 
objs = torch.LongTensor(n_objs).fill_(-1)
obj_idx_mapping = {}
for i, obj_idx in enumerate(obj_idxs):
    objs[i] = vg.data['object_names'][index, obj_idx].item()
    obj_idx_mapping[obj_idx] = i

triples = []
print('relationships_per_image', vg.data['relationships_per_image'][index].item())
for r_idx in range(vg.data['relationships_per_image'][index].item()):
    s = vg.data['relationship_subjects'][index, r_idx].item()
    p = vg.data['relationship_predicates'][index, r_idx].item()
    o = vg.data['relationship_objects'][index, r_idx].item()
    s = obj_idx_mapping.get(s, None)
    o = obj_idx_mapping.get(o, None)
    if s is not None and o is not None:
        triples.append([s, p, o])

triples = torch.LongTensor(triples)
objs, obj_bbox, triples, len(objs)

In [ ]:
import PIL.Image

img = np.array(PIL.Image.open(img_path))
boxes = obj_bbox
labels = [vg.vocab['object_idx_to_name'][o] for o in objs]
print(labels)

In [ ]:
objs, boxes, triples = vg[10]
print(len(objs), len(boxes), len(triples))
objs, boxes, triples


In [ ]:
for s,p,o in triples:
    s = vg.vocab['object_idx_to_name'][s]
    p = vg.vocab['pred_idx_to_name'][p]
    o = vg.vocab['object_idx_to_name'][o]
    print(f"{s} -> {p} -> {o}")

In [ ]:
import json
path = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/attributes.json'
with open(path, 'r') as f:
    data = json.load(f) 
print(data)


In [ ]:
import h5py
path =  '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/train.h5'
with h5py.File(path, 'r') as hdf:
    data_fnames = sorted(hdf.keys())
    # for k in data_fnames:
        # print(k, hdf[k][:1])
    # obj_boxes = hdf['object_boxes'][:]
    # obj_names = hdf['object_names'][:]

    

In [ ]:
obj_names

In [ ]:
data_fnames

In [ ]:
path = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/objects.json'
with open(path, 'r') as f:
    data = json.load(f) 
print(data[0])

In [ ]:
path = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/scene_graphs.json'
with open(path, 'r') as f:
    data = json.load(f) 
print(data[0])

In [ ]:
print(data[0].keys())

In [ ]:
data[0]['relationships']

In [ ]:
data[0]['image_id']
data[0]['regions']